# Solution creater template 

This notebook assumes that https://co2020-ba-vu.challenges.ortec.com/zips/Validator.zip has been unpacked to a subdirectory of the current directory called `ParticipantsSuite`, changes to a different location are obvious. 

It also assumes a subdirectory `instances` containing the instance files. 

In [1]:
import sys
import os
import pandas as pd
sys.path.append('./ParticipantsSuite')

from InstanceCVRPTWUI import InstanceCVRPTWUI
instance = InstanceCVRPTWUI('instances/ORTEC_Test_01.txt')

In [2]:
if instance.isValid():
    instance.calculateDistances()
    import numpy
    nLoc = len(instance.Coordinates)
    d_df = pd.DataFrame([ [ instance.calcDistance[i][j] for j in range(nLoc) ] for i in range(nLoc) ])

t_df = pd.DataFrame([(i.ID, i.weight, i.amount, i.cost) for i in instance.Tools], columns=['ID','weight','amount','cost']).set_index('ID')
r_df = pd.DataFrame([(i.ID, i.node, i.fromDay, i.toDay, i.numDays, i.tool, i.toolCount) for i in instance.Requests], columns=['ID','node','fromDay','toDay','numDays','tool','toolCount']).set_index('ID')
c_df = pd.DataFrame([(i.ID, i.X, i.Y) for i in instance.Coordinates], columns=['ID','X','Y']).set_index('ID')

## List of DataFrames with data:

Tools: `t_df`, Requests: `r_df`, Coordinates: `c_df`, Distances: `d_df`

In [ ]:
# Solution....